In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re
from PIL import Image

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MODELS = {
    "UI-TARS 7B SFT": "bytedance-research/UI-TARS-7B-SFT"
}

In [ ]:
def load_model(model_path):
    """Load model and processor with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")

    """Load model and processor."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token
    )
    processor = AutoProcessor.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, processor

In [ ]:
def evaluate_vqa(model, processor, dataset_name=""):
    """Evaluate on VQAv2 dataset for Visual Question Answering."""
    dataset = load_dataset(dataset_name, split="validation[:5]", trust_remote_code=True)
    metric = load_metric("squad_v2")

    for example in dataset:
        image_path = example["image"]
        image = Image.open(image_path).convert("RGB")

        question = example["question"]
        input_text = f"Q: {question} A:"

        inputs = processor(images=image, text=input_text, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=processor.eos_token_id)
        prediction = processor.decode(outputs[0], skip_special_tokens=True).strip()

        correct_answer = example["answers"]["text"]

        metric.add(predictions={"id": example["question_id"], "prediction_text": prediction},
                   references={"id": example["question_id"], "answers": example["answers"]})

        print("\n🖼️ Image:", image_path)
        print("❓ Question:", question)
        print("✅ Ground Truth Answer:", correct_answer)
        print("🤖 Model Prediction:", prediction)

    return metric.compute()

In [ ]:
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, processor = load_model(model_path)

    print("Squad QA:", evaluate_vqa(model, processor))


Evaluating UI-TARS 7B SFT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

NameError: name 'evaluate_math' is not defined